In [1]:
import sys
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from copy import deepcopy
from tqdm import tqdm
import random
from random import shuffle

In [2]:
import cv2
import numpy as np
import random
import os
import re
from tqdm import tqdm
from copy import deepcopy
import sys

import cv2
import numpy as np
from tqdm import tqdm
from multiprocessing import Process, Manager


In [3]:
from IPython.display import clear_output
from matplotlib import pyplot as plt
import collections
%matplotlib inline

def live_plot(data, figsize=(16,8), title=''):
    clear_output(wait=True)
    plt.figure(figsize=figsize)
    plt.imshow(convert_to_rgb(data.astype(int)))
    plt.title(title)
    plt.show()

In [4]:
manager = Manager()  # create only 1 mgr
# prob_dict = manager.dict()

## 读取数据

In [5]:
def convert_to_rgb(img):
    img = deepcopy(img)
    b,g,r = cv2.split(img)  
    img = cv2.merge([r,g,b])
    return img

In [6]:
background = cv2.imread('../../base_data/background.jpeg')
hedge = cv2.imread('../../base_data/hedge.jpeg')

## 生成视频代码

In [7]:
def generate_video(frame_dir=None, video_dir=None, video_name='video.avi'):

    image_num = len([i for i in os.listdir(frame_dir) if '.jpg' in i])
    image_name_list = [f'{i}.jpg' for i in range(image_num)]
    frame = cv2.imread(os.path.join(frame_dir, image_name_list[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(f'{video_dir}/{video_name}', 0, 20, (width, height))

    # video = cv2.VideoWriter(f'{self.video_dir}/{video_name}', 0, 20, (width, height))

    for image in image_name_list:
        video.write(cv2.imread(os.path.join(frame_dir, image)))

    cv2.destroyAllWindows()
    video.release()

## 定义参数

In [8]:
speed_dict = {
    "background": 1,
    "hedge_1": 2,
    "hedge_2": 3,
    "black_bar": 4
}

crop_height = 900
crop_width = 1500
slide_width = 800
camera_width = 250

crop_width_extend = crop_width + 1000

size_1 = camera_width * 2
size_2 = slide_width

dimension = 3
black_bar_width = 10

background_prob = 0.7
noise_average_size = 10

vertical_movement = [0]
horizontal_movement = [1, 2]

In [9]:
background_crop = deepcopy(background[-crop_height:, :crop_width, :])
# live_plot(background_crop)

In [10]:
hedge_crop = deepcopy(hedge[-crop_height:, :crop_width, :])
# live_plot(hedge_crop)

In [11]:
def rgb_mean(image):
    return tuple([np.mean([np.mean(image[:, :, i])]) for i in range(2, -1, -1)])

def initialise_image(image):
    reshape_size = image.shape[0] * image.shape[1]
    a = np.zeros((dimension, reshape_size))
    for i in range(3):
        a[i] = np.array(image[:, :, i]).reshape(1, reshape_size)
    return a

def random_select():
    return random.randint(0, 250), random.randint(0, 750)

def random_radius():
    return random.randint(10, 35)

def mahalanobis(x, mean, cov):
    return np.sqrt(np.linalg.inv(cov).dot(np.transpose(x - mean)).dot((x - mean)))

def cal_ratio_prob(x):
    return 1 - (x/x.sum())

def ratio_soft_max(self, x):
    return ratio_prob(soft_max(x))

def soft_max(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [12]:
hedge_mean = np.array(rgb_mean(hedge_crop))
background_mean = np.array(rgb_mean(background_crop))
hedge_cov = np.cov(initialise_image(hedge_crop))
background_cov = np.cov(initialise_image(background_crop))

## 显示图片

In [13]:
# plt.imshow(convert_to_rgb(background))
# plt.show()
# plt.imshow(convert_to_rgb(hedge))
# plt.show()

In [14]:
background.shape, hedge.shape

## 定义参数

In [15]:
speed_dict = {
    "background": 1,
    "hedge_1": 2,
    "hedge_2": 3,
    "black_bar": 4
}

crop_height = 900
crop_width = 1500
slide_width = 800
camera_width = 250

crop_width_extend = crop_width + 1000

size_1 = camera_width * 2
size_2 = slide_width

dimension = 3
black_bar_width = 10

background_prob = 0.7
noise_average_size = 10

vertical_movement = [0]
horizontal_movement = [1, 2]

In [16]:
background_crop = deepcopy(background[-crop_height:, :crop_width, :])
# live_plot(background_crop)

In [17]:
hedge_crop = deepcopy(hedge[-crop_height:, :crop_width, :])
# live_plot(hedge_crop)

## 检测位移的算法

In [18]:
def calculate_frame_prob(frame):
    frame_prob = np.zeros(frame.shape)
    for i in range(frame.shape[0]):
        for j in range(frame.shape[1]):
            pixel = frame[i][j]
            hedge_distance = mahalanobis(pixel, hedge_mean, hedge_cov)
            top_distance = mahalanobis(pixel, background_mean, background_cov)
            ratio_prob = cal_ratio_prob(np.array([top_distance, hedge_distance]))[0]
            frame_prob[i][j] = round(ratio_prob)
    return frame_prob

In [19]:
def calculate_accurate(num_list, scene_dict, accurate_dict):
    for num in tqdm(num_list, ncols=70):
        compare_list = []
        scene_num = random.choice(range(min(scene_dict.keys()), max(scene_dict.keys())-1))
        for num in [scene_num, scene_num + 1]:
            file_name = scene_dict[num]['file_name']
            verti_pos = scene_dict[num]['verti_pos']
            hori_pos = scene_dict[num]['hori_pos']
            frame = np.array(cv2.imread(f"{frame_dir}/{file_name}"))
            compare_list.append([frame, (verti_pos, hori_pos), num])
        ground_truth = np.array(compare_list[1][1]) - np.array(compare_list[0][1])

        compare_dict = {}
        frame = compare_list[0][0]
        compare_frame = compare_list[1][0]
        for _ in range(32):
            break_flag = 1
            for _ in range(128):
                frame_i, frame_j = random.choice(range(frame.shape[0])), random.choice(range(frame.shape[1]))
                pixels = frame[frame_i - n:frame_i + n, frame_j - n:frame_j + n, :]
                pixels_prob = calculate_frame_prob(pixels)
                if pixels.shape == set_shape and np.sum(pixels_prob) >= set_shape[0] * set_shape[1] * set_shape[2] * 0.95:
                    break_flag = 0
                    break
            if break_flag:
                compare_dict = {}
                print('break')
                break

            for i in range(frame_i-m, frame_i+m):
                for j in range(frame_j-m, frame_j+m):
                    compare_pixels = compare_frame[i - n:i + n,j - n:j + n,:]
                    if pixels.shape == set_shape and compare_pixels.shape == set_shape:
                        diff_array = np.array([i for i in ((pixels - compare_pixels)).reshape(-1,1) if i != 0])
                        diff = np.sum(np.power(diff_array,2))/len(diff_array)
                        compare_dict[(frame_i - i, frame_j - j)] = compare_dict.get((frame_i - i, frame_j - j), []) + [diff]
        compare_dict = {k:np.mean(v) for k,v in compare_dict.items()}
        compare_dict = {k: v for k, v in sorted(compare_dict.items(), key=lambda item: item[1])}
        if list(compare_dict.keys()):
            predict = np.array(list(compare_dict.keys())[0])
            accurate_dict[num] = list(ground_truth) == list(predict)
    
    

In [20]:
def split_list(alist, wanted_parts=1):
    length = len(alist)
    return [ alist[i*length // wanted_parts: (i+1)*length // wanted_parts] 
             for i in range(wanted_parts) ]

In [21]:
manager = Manager()

In [ ]:
m = 10
accuracy_dict = {}
for sn in tqdm(range(100,1,-1), ncols=70):
    frame_dir = '../scene_data/{}'.format(sn)
    
    scene_dict = {}
    real_back_prob_list = []
    for file_name in os.listdir(frame_dir):
        if file_name[0] != '.':
            num, verti_pos, hori_pos, real_back_prob = file_name.replace('.jpg', '').split('_')
            real_back_prob_list.append(int(real_back_prob))
            scene_dict[int(num)] = {'file_name': file_name, 'num':num, 'verti_pos':int(verti_pos), 'hori_pos':int(hori_pos)}
    real_back_prob_mean = int(np.mean(real_back_prob_list))
    accuracy_dict[real_back_prob_mean] = {}
    
    for n in tqdm([8,4,2,1], ncols=70):
        set_shape = (2*n, 2*n, 3)
        result_compare_list = []
        accurate_dict = manager.dict()
        jobs = []
        
        for num_list in split_list(list(range(100)), 16):
            p = Process(target=calculate_accurate, args=(num_list, scene_dict, accurate_dict))
            jobs.append(p)
            p.start()
            
        for proc in tqdm(jobs, ncols=70):
            proc.join()
            
        result_compare_list = accurate_dict.values()
        accuracy = np.nan
        if len(result_compare_list):
            accuracy = sum(result_compare_list)*100/len(result_compare_list)
        accuracy_dict[real_back_prob_mean][n] = accuracy
        df = pd.DataFrame().from_dict(accuracy_dict)
        df.to_csv('accuracy_2.csv', index=True)



 17%|█████▊                             | 1/6 [00:15<01:18, 15.61s/it]

 14%|█████                              | 1/7 [00:15<01:34, 15.81s/it]

 17%|█████▊                             | 1/6 [00:15<01:19, 15.90s/it]

 17%|█████▊                             | 1/6 [00:15<01:19, 15.88s/it]

 17%|█████▊                             | 1/6 [00:15<01:19, 15.96s/it]

 17%|█████▊                             | 1/6 [00:16<01:20, 16.01s/it]

 17%|█████▊                             | 1/6 [00:15<01:19, 15.97s/it]

 14%|█████                              | 1/7 [00:15<01:34, 15.81s/it]

 14%|█████                              | 1/7 [00:16<01:37, 16.21s/it]

 17%|█████▊                             | 1/6 [00:16<01:21, 16.25s/it]

 14%|█████                              | 1/7 [00:16<01:37, 16.32s/it]

 17%|█████▊                             | 1/6 [00:16<01:21, 16.36s/it]

 17%|█████▊                             | 1/6 [00:16<01:22, 16.41s/it]

 17%|█████▊                             | 1/6 [00:16<01:21, 16

 25%|████████▌                         | 1/4 [01:52<05:38, 112.69s/it]

  0%|                                           | 0/6 [00:00<?, ?it/s]

  0%|                                           | 0/6 [00:00<?, ?it/s]

  0%|                                           | 0/6 [00:00<?, ?it/s]

  0%|                                           | 0/7 [00:00<?, ?it/s]

  0%|                                           | 0/6 [00:00<?, ?it/s]

  0%|                                           | 0/6 [00:00<?, ?it/s]

  0%|                                           | 0/7 [00:00<?, ?it/s]

  0%|                                           | 0/6 [00:00<?, ?it/s]

  0%|                                           | 0/6 [00:00<?, ?it/s]

  0%|                                           | 0/6 [00:00<?, ?it/s]

  0%|                                          | 0/16 [00:00<?, ?it/s]

  0%|                                           | 0/6 [00:00<?, ?it/s]

  0%|                                           | 0/6 [00:00<?, 

100%|███████████████████████████████████| 6/6 [00:26<00:00,  4.48s/it]


 86%|██████████████████████████████     | 6/7 [00:27<00:04,  4.46s/it]

100%|███████████████████████████████████| 6/6 [00:26<00:00,  4.46s/it]


100%|███████████████████████████████████| 6/6 [00:26<00:00,  4.50s/it]


 86%|██████████████████████████████     | 6/7 [00:27<00:04,  4.52s/it]

100%|███████████████████████████████████| 6/6 [00:27<00:00,  4.54s/it]


 86%|██████████████████████████████     | 6/7 [00:27<00:04,  4.50s/it]

100%|███████████████████████████████████| 6/6 [00:27<00:00,  4.53s/it]


100%|███████████████████████████████████| 6/6 [00:27<00:00,  4.59s/it]


100%|███████████████████████████████████| 6/6 [00:27<00:00,  4.58s/it]


 86%|██████████████████████████████     | 6/7 [00:28<00:04,  4.69s/it]

100%|███████████████████████████████████| 7/7 [00:31<00:00,  4.50s/it]


100%|███████████████████████████████████| 7/7 [00:31<00:00,  4.51s/it]


 25%|████████▌                         | 4/16 [00:31<03

 83%|█████████████████████████████▏     | 5/6 [00:06<00:01,  1.33s/it]

 83%|█████████████████████████████▏     | 5/6 [00:06<00:01,  1.32s/it]

 83%|█████████████████████████████▏     | 5/6 [00:06<00:01,  1.32s/it]

 83%|█████████████████████████████▏     | 5/6 [00:06<00:01,  1.28s/it]

 83%|█████████████████████████████▏     | 5/6 [00:06<00:01,  1.31s/it]

 71%|█████████████████████████          | 5/7 [00:06<00:02,  1.33s/it]

 83%|█████████████████████████████▏     | 5/6 [00:06<00:01,  1.34s/it]

 83%|█████████████████████████████▏     | 5/6 [00:06<00:01,  1.33s/it]

 71%|█████████████████████████          | 5/7 [00:06<00:02,  1.38s/it]

 71%|█████████████████████████          | 5/7 [00:06<00:02,  1.35s/it]

 83%|█████████████████████████████▏     | 5/6 [00:07<00:01,  1.40s/it]

 83%|█████████████████████████████▏     | 5/6 [00:07<00:01,  1.44s/it]

 83%|█████████████████████████████▏     | 5/6 [00:07<00:01,  1.48s/it]

100%|███████████████████████████████████| 6/6 [00:07<00:00,  1.3

 57%|████████████████████               | 4/7 [00:02<00:01,  1.93it/s]

 67%|███████████████████████▎           | 4/6 [00:02<00:01,  1.87it/s]

 67%|███████████████████████▎           | 4/6 [00:02<00:01,  1.83it/s]

 67%|███████████████████████▎           | 4/6 [00:02<00:01,  1.77it/s]

 67%|███████████████████████▎           | 4/6 [00:02<00:01,  1.82it/s]

 57%|████████████████████               | 4/7 [00:02<00:01,  1.85it/s]

 67%|███████████████████████▎           | 4/6 [00:02<00:01,  1.85it/s]

 43%|███████████████                    | 3/7 [00:02<00:02,  1.57it/s]

 67%|███████████████████████▎           | 4/6 [00:02<00:01,  1.89it/s]

 57%|████████████████████               | 4/7 [00:02<00:01,  1.82it/s]

 67%|███████████████████████▎           | 4/6 [00:02<00:01,  1.67it/s]

 67%|███████████████████████▎           | 4/6 [00:02<00:01,  1.64it/s]

 67%|███████████████████████▎           | 4/6 [00:02<00:01,  1.65it/s]

 67%|███████████████████████▎           | 4/6 [00:02<00:01,  1.7

...

  0%|                                           | 0/6 [00:00<?, ?it/s]

  0%|                                           | 0/6 [00:00<?, ?it/s]

  0%|                                           | 0/6 [00:00<?, ?it/s]

  0%|                                           | 0/6 [00:00<?, ?it/s]

  0%|                                           | 0/7 [00:00<?, ?it/s]

  0%|                                           | 0/7 [00:00<?, ?it/s]

  0%|                                           | 0/6 [00:00<?, ?it/s]

  0%|                                           | 0/7 [00:00<?, ?it/s]

  0%|                                          | 0/16 [00:00<?, ?it/s]

 17%|█████▊                             | 1/6 [00:04<00:21,  4.27s/it]

 17%|█████▊                             | 1/6 [00:04<00:22,  4.43s/it]

 17%|█████▊                             | 1/6 [00:04<00:22,  4.44s/it]

 17%|█████▊                             | 1/6 [00:04<00:21,  4.34s/it]

 17%|█████▊                             | 1/6 [00:04<00:22,  4.4

100%|███████████████████████████████████| 6/6 [00:26<00:00,  4.35s/it]


100%|███████████████████████████████████| 6/6 [00:25<00:00,  4.32s/it]


 86%|██████████████████████████████     | 6/7 [00:26<00:04,  4.38s/it]

100%|███████████████████████████████████| 6/6 [00:26<00:00,  4.41s/it]


100%|███████████████████████████████████| 7/7 [00:29<00:00,  4.24s/it]


100%|███████████████████████████████████| 7/7 [00:29<00:00,  4.25s/it]


100%|███████████████████████████████████| 7/7 [00:29<00:00,  4.27s/it]


100%|███████████████████████████████████| 7/7 [00:30<00:00,  4.36s/it]


100%|█████████████████████████████████| 16/16 [00:30<00:00,  1.89s/it]

 50%|█████████████████▌                 | 2/4 [02:17<02:48, 84.28s/it]

  0%|                                           | 0/6 [00:00<?, ?it/s]

  0%|                                           | 0/6 [00:00<?, ?it/s]

  0%|                                           | 0/6 [00:00<?, ?it/s]

  0%|                                           | 0/7 [00

 57%|████████████████████               | 4/7 [00:05<00:04,  1.39s/it]

 67%|███████████████████████▎           | 4/6 [00:06<00:02,  1.47s/it]

 71%|█████████████████████████          | 5/7 [00:06<00:02,  1.27s/it]

 83%|█████████████████████████████▏     | 5/6 [00:06<00:01,  1.26s/it]

/home/yf/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in double_scalars
 83%|█████████████████████████████▏     | 5/6 [00:06<00:01,  1.29s/it]

 83%|█████████████████████████████▏     | 5/6 [00:06<00:01,  1.30s/it]

 83%|█████████████████████████████▏     | 5/6 [00:06<00:01,  1.30s/it]

 83%|█████████████████████████████▏     | 5/6 [00:06<00:01,  1.30s/it]

 83%|█████████████████████████████▏     | 5/6 [00:06<00:01,  1.32s/it]

 71%|█████████████████████████          | 5/7 [00:06<00:02,  1.31s/it]

 83%|█████████████████████████████▏     | 5/6 [00:06<00:01,  1.30s/it]

 83%|█████████████████████████████▏     | 5/6 [00:06<00:01,  1.34s/it]

 83



 33%|███████████▋                       | 2/6 [00:01<00:01,  2.08it/s]

 33%|███████████▋                       | 2/6 [00:01<00:01,  2.05it/s]

/home/yf/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in double_scalars
 29%|██████████                         | 2/7 [00:01<00:02,  1.98it/s]/home/yf/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in double_scalars


 33%|███████████▋                       | 2/6 [00:01<00:02,  1.80it/s]

 33%|███████████▋                       | 2/6 [00:01<00:02,  1.68it/s]/home/yf/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in double_scalars


 29%|██████████                         | 2/7 [00:01<00:03,  1.63it/s]

 29%|██████████                         | 2/7 [00:01<00:02,  1.77it/s]

 33%|███████████▋                       | 2/6 [00:01<00:02,  1.77it/s]

 33%|███████████▋      

  0%|                                           | 0/6 [00:00<?, ?it/s]

  0%|                                           | 0/6 [00:00<?, ?it/s]

  0%|                                           | 0/6 [00:00<?, ?it/s]

  0%|                                           | 0/7 [00:00<?, ?it/s]

  0%|                                           | 0/6 [00:00<?, ?it/s]

  0%|                                           | 0/6 [00:00<?, ?it/s]

 17%|█████▊                             | 1/6 [00:15<01:17, 15.55s/it]

 14%|█████                              | 1/7 [00:15<01:34, 15.69s/it]

 17%|█████▊                             | 1/6 [00:15<01:18, 15.79s/it]

 17%|█████▊                             | 1/6 [00:15<01:18, 15.77s/it]

 14%|█████                              | 1/7 [00:16<01:36, 16.06s/it]

 14%|█████                              | 1/7 [00:16<01:38, 16.34s/it]

 17%|█████▊                             | 1/6 [00:16<01:21, 16.38s/it]

 17%|█████▊                             | 1/6 [00:16<01:21, 16.4

In [ ]:
# def split_list(alist, wanted_parts=1):
#     length = len(alist)
#     return [ alist[i*length // wanted_parts: (i+1)*length // wanted_parts] 
#              for i in range(wanted_parts) ]

# A = [0,1,2,3,4,5,6,7,8,9]

# print(split_list(A, wanted_parts=1))
# print(split_list(A, wanted_parts=2))
# print(split_list(A, wanted_parts=8))